In [ ]:
import pandas as pd
from snowflake.snowpark import Session
from snowflake.snowpark.functions import udf
from snowflake.snowpark.types import DecimalType
import os
from dotenv import load_dotenv

In [ ]:
#Load Environment Variables
load_dotenv()

In [ ]:
#Setting up credentials

account = os.getenv('account')
user = os.getenv('user')
password = os.getenv('password')
role = os.getenv('role')
warehouse = os.getenv('warehouse')
database = os.getenv('database')
schema = os.getenv('schema')
stage = os.getenv('stage')

In [ ]:
#Connecting to snowflake
connection_parameters = {
"account": account,
"user": user,
"password": password,
"role": role,
"warehouse": warehouse,
"database": database,
"schema": schema
 }

#Creating a snowflake session
session = Session.builder.configs(connection_parameters).create()

In [ ]:
def minus_one(x: float) -> float:
  return x-1

In [ ]:
session.udf.register(
    func = minus_one
  , return_type = DecimalType(20,2)
  , input_types = [DecimalType(20,2)]
  , is_permanent = True
  , name = 'minus_one'
  , replace = True
  , stage_location = stage
 )

In [ ]:
#Using SQL with snowpark session
session.sql("""SELECT I_CURRENT_PRICE, MINUS_ONE(I_CURRENT_PRICE) 
                FROM SNOWFLAKE_SAMPLE_DATA.TPCDS_SF100TCL.ITEM""").show()